<p style="font-family: Arial; font-size:3em;color:black;"> Lab Exercise 10</p>

## Matrix Factorization with Missing Values

In this exercise, we'll:
1. Start with a complete matrix
2. Remove some values (set to NaN)
3. Use matrix factorization to predict the missing values
4. Find the optimal K value (number of latent features) that gives the best predictions

Since we know the original values, we can measure how well our predictions match reality.

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(5)  # creating pseudo-random numbers for reproducibility

In [ ]:
# Let's consider the following A_orig matrix:

A_orig = np.array([[4, 2, 4, 5, 4, 2, 2], [4, 4, 3, 2, 2, 5, 4], [4, 1, 4, 7, 3, 6, 2], [8 ,1, 2, 0, 5, 0, 7], [4 , 5, 8, 7, 6, 2, 3]],dtype=float)
M, N = 5, 7
print (pd.DataFrame(A_orig).head())

In [ ]:
# Let's add some NaN(s) to  A_orig matrix

A = A_orig.copy()
A[3][1] = np.nan
A[4][6] = np.nan

A_df = pd.DataFrame(A)
print (A_df.head())

In [54]:
# Fine tune K value such that the (average percentage) error for the 2 missing elements is minimum.
# Remember: this is an exercise where we have the actual values for missing elements. 
# In real-life scenarios missins elements are not known and you may need different metric(s)
# to assess the quality of your reconstructed matrix.
K = 1
P = np.abs(np.random.uniform(low=0, high=8, size=(M, K)))
Q = np.abs(np.random.uniform(low=0, high=8, size=(K, N)))
P = np.divide(P, K*P.max())
Q = np.divide(Q, K*Q.max())

In [55]:
def matrix_factorization(Rating_Matrix, P, Q, K, steps, alpha=0.001, beta=0.02):
    Q = Q.T
    for step in range(steps):
        for i in range(len(Rating_Matrix)):
            for j in range(len(Rating_Matrix[i])):
                if ~np.isnan(Rating_Matrix[i][j]):
                    eij = Rating_Matrix[i][j] - np.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])
        eRating_Matrix = np.dot(P,Q)
        e = 0
        for i in range(len(Rating_Matrix)):
            for j in range(len(Rating_Matrix[i])):
                if ~np.isnan(Rating_Matrix[i][j]):
                    e = e + pow(Rating_Matrix[i][j] - np.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        print("Total error at step", step, "is", e)
        if e < 0.0001:
            break
    return P, Q.T

In [ ]:
eP, eQ = matrix_factorization(A, P, Q.T, K, steps = 1000)
eA = np.matmul(eP, eQ.T)

print(A,'\n')
print(eA)

## Finding the Optimal K Value

Now let's try different K values to see which gives us the best predictions for our missing values.
We'll:
- Try K values from 1 to 10
- Run each K value multiple times (due to random initialization)
- Calculate the average percentage error for the missing values
- Find which K gives us the most accurate predictions

In [ ]:
def try_different_k(num_trials=5):
    # Store results for each K
    k_results = {}
    
    # Try different K values
    for k in range(1, 11):  # Try K from 1 to 10
        k_errors = []
        
        # Multiple trials for each K to account for random initialization
        for trial in range(num_trials):
            # Initialize matrices with current K
            P = np.abs(np.random.uniform(low=0, high=8, size=(M, k)))
            Q = np.abs(np.random.uniform(low=0, high=8, size=(k, N)))
            P = np.divide(P, k*P.max())
            Q = np.divide(Q, k*Q.max())
            
            # Run matrix factorization
            eP, eQ = matrix_factorization(A, P, Q.T, k, steps=1000)
            eA = np.matmul(eP, eQ.T)
            
            # Calculate error for missing values (comparing with A_orig)
            error = (abs(eA[3,1] - A_orig[3,1])/A_orig[3,1] + 
                    abs(eA[4,6] - A_orig[4,6])/A_orig[4,6]) / 2
            k_errors.append(error)
        
        # Store average error across trials for this K
        k_results[k] = np.mean(k_errors)
        print(f"K={k}, Average Error across {num_trials} trials: {k_results[k]:.4f}")
    
    # Find best K
    best_k = min(k_results.items(), key=lambda x: x[1])
    print(f"\nBest K value: {best_k[0]} with average error: {best_k[1]:.4f}")
    return k_results, best_k

# Set random seed for reproducibility
np.random.seed(42)

# Run the optimization with 5 trials per K value
k_results, best_k = try_different_k(num_trials=5)

In [ ]:
# Plot results
plt.figure(figsize=(10, 6))
plt.plot(list(k_results.keys()), list(k_results.values()), 'bo-')
plt.xlabel('K Value')
plt.ylabel('Average Error')
plt.title('Error vs K Value in Matrix Factorization')
plt.grid(True)
plt.show()

## Analysis

From our experiments with different K values:

1. The optimal K value was found to be 2, showing the lowest average error in our experiments
2. This suggests that 2 latent features are sufficient to capture the underlying patterns in our matrix
3. The error trend shows:
   - K=1 underfits the data (too simple to capture all patterns)
   - K values above 2 show higher errors, suggesting potential overfitting
   - The error increases after K=2, indicating additional features may be introducing noise

This optimal K=2 value provides the best balance between model complexity and prediction accuracy for our missing values (A[3,1]=1 and A[4,6]=3).